In [49]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [9]:
p = pd.read_csv('petr.csv')
t = pd.read_csv('tema.csv')
z = pd.read_excel('zaven.xlsx')
o = pd.read_csv('olya.csv')
s = pd.read_csv('sergey.csv')
p2 = pd.read_csv('baseline.csv')

In [83]:
p = p.sort_values('suspicious', ascending=False)

In [84]:
t = t.sort_values('value', ascending=False)

In [85]:
z = z.sort_values('predict', ascending=False)

In [86]:
o = o.sort_values('1', ascending=False)

In [87]:
p2 = p2.sort_values('suspicious', ascending=False)

In [88]:
print(
    np.corrcoef(p.hash[:277], p2.hash[:277]),
    '\n',
    np.corrcoef(p.hash[:277], t.hash),
    '\n',
    np.corrcoef(p.hash[:277], z.driver),
    '\n',
    np.corrcoef(t.hash, z.driver),
    '\n',
    np.corrcoef(p.hash[:277], o['driver hash']),
#     np.corrcoef(p.hash, p2.hash),
#     np.corrcoef(p.hash, p2.hash),
)

[[ 1.         -0.05746491]
 [-0.05746491  1.        ]] 
 [[ 1.         -0.02516865]
 [-0.02516865  1.        ]] 
 [[ 1.        -0.0173215]
 [-0.0173215  1.       ]] 
 [[ 1.         -0.02878422]
 [-0.02878422  1.        ]] 
 [[1.         0.06872582]
 [0.06872582 1.        ]]


In [89]:
fin_df = p.rename(columns={'suspicious':'sus_0'})

for df in [t.rename(columns={'value':'sus_1'}), 
           o.rename(columns={'1':'sus_2', 'driver hash': 'hash'}), 
           z.rename(columns={'predict':'sus_3', 'driver': 'hash'}), 
           p2.rename(columns={'suspicious':'sus_4'})]:
    #res.append(StandardScaler().fit_transform(np.array([df.suspicious]).T).T[0][:277])
    fin_df = fin_df.merge(df, left_on='hash', right_on='hash')

fin_df = fin_df[['hash', 'sus_0', 'sus_1', 'sus_2', 'sus_3', 'sus_4']]
fin_df[['sus_0', 'sus_1', 'sus_2', 'sus_3', 'sus_4']] = MinMaxScaler().fit_transform(fin_df[['sus_0', 'sus_1', 'sus_2', 'sus_3', 'sus_4']])

In [90]:
fin_df

,hash,sus_0,sus_1,sus_2,sus_3,sus_4
0,-7021406106444285482,1.0000,0.066570,0.075843,1.000000,0.889549
1,3357736502633273445,1.0000,0.218391,0.096240,0.985714,0.882822
2,6971259077322903056,1.0000,0.228769,0.166213,0.971429,0.898067
3,-3896471437951454158,1.0000,0.258565,0.218937,0.985714,0.873814
4,-1007930254617779546,0.9875,0.135994,0.020044,0.914286,0.860624
...,...,...,...,...,...,...
272,2923372506500238361,0.0000,0.083894,0.016515,0.600000,0.167777
273,-8787660130012666034,0.0000,0.160681,0.196681,0.542857,0.092609
274,7799079856941240657,0.0000,0.222523,0.018739,0.142857,0.067820
275,8708539809027958552,0.0000,0.038739,0.059158,0.028571,0.063821


In [112]:
fin_df['sus'] = fin_df[['sus_0', 'sus_1', 'sus_4']].mean(axis=1)
fin_df

,hash,sus_0,sus_1,sus_2,sus_3,sus_4,sus
0,-7021406106444285482,1.0000,0.066570,0.075843,1.000000,0.889549,0.652040
1,3357736502633273445,1.0000,0.218391,0.096240,0.985714,0.882822,0.700404
2,6971259077322903056,1.0000,0.228769,0.166213,0.971429,0.898067,0.708945
3,-3896471437951454158,1.0000,0.258565,0.218937,0.985714,0.873814,0.710793
4,-1007930254617779546,0.9875,0.135994,0.020044,0.914286,0.860624,0.661373
...,...,...,...,...,...,...,...
272,2923372506500238361,0.0000,0.083894,0.016515,0.600000,0.167777,0.083890
273,-8787660130012666034,0.0000,0.160681,0.196681,0.542857,0.092609,0.084430
274,7799079856941240657,0.0000,0.222523,0.018739,0.142857,0.067820,0.096781
275,8708539809027958552,0.0000,0.038739,0.059158,0.028571,0.063821,0.034187


In [113]:
pred = fin_df[['hash', 'sus']].set_index('hash').sort_values('sus', ascending=False).rename(columns={'sus':'suspicious'})

In [116]:
pred

,suspicious
hash,
-7860657497081293425,0.782391
-8478910855025982897,0.769308
3421995393814356075,0.718762
-3896471437951454158,0.710793
6971259077322903056,0.708945
...,...
2710587379068018652,0.025541
-4016388014455777024,0.024668
6516912822513505159,0.023012


In [117]:
pred.to_csv('res2.csv')